<a href="https://colab.research.google.com/github/mahidedhia/ResumeParser-NLP/blob/master/ResumeParsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [12]:
!pip install pdfminer.six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
filename = "QA-Manual-Tester.pdf"

In [14]:
#temporary method to extract from pdf
from pdfminer.high_level import extract_text

def pdfExtract(file):
  text = extract_text(file)
  return text

In [15]:
text = pdfExtract(filename)

PARSING THE RESUME TEXT OBTAINED FROM PDF

In [16]:
headers_experience = (
        'career profile',
        'employment history',
        'work history',
        'work experience',
        'experience',
        'professional experience',
        'professional background',
        'additional experience',
        'career related experience',
        'related experience',
        'programming experience',
        'freelance',
        'freelance experience',
        'army experience',
        'military experience',
        'military background',
)
headers_education = (
        'academic background',
        'academic experience',
        'programs',
        'courses',
        'related courses',
        'education',
        'qualifications',
        'educational background',
        'educational qualifications',
        'educational training',
        'education and training',
        'training',
        'academic training',
        'professional training',
        'course project experience',
        'related course projects',
        'internship experience',
        'internships',
        'apprenticeships',
        'college activities',
        'certifications',
        'special training',
    )
headers_skills = (
        'credentials',
        'areas of experience',
        'areas of expertise',
        'areas of knowledge',
        'skills',
        "other skills",
        "other abilities",
        'career related skills',
        'professional skills',
        'specialized skills',
        'technical skills',
        'computer skills',
        'personal skills',
        'computer knowledge',        
        'technologies',
        'technical experience',
        'proficiencies',
        'languages',
        'language competencies and skills',
        'programming languages',
        'competencies'
    )
headers_projects = (
    'projects',
    'personal projects',
    'academic projects',
)

In [17]:
# A complete sentence contains at least one subject, one predicate, one object, and closes with punctuation. 
# Subject and object are almost always nouns, and the predicate is always a verb.
def checkSentence(text):
  text=nlp(text)
  has_noun = 2
  has_verb = 1
  for token in text:
    if token.pos_ in ["NOUN", "PROPN", "PRON"]:
        has_noun -= 1
    elif token.pos_ == "VERB":
        has_verb -= 1
  if has_noun < 1 and has_verb < 1:
    return True
  return False

In [20]:
class ResumeParser:
  def __init__(self, text):
    self.resume_text = text
    self.lines = text.split("\n")
  
  resume_sections = {
      'name': {},
      'experience': {},
      'education': {},
      'skills': {},
      'projects': {},
  }
  header_indices = {}

  #Obtain line nos of Headers 
  def getHeaderIndices(self):
    for i, line in enumerate(self.lines):
      if len(line):
        #Header always starts in Uppercase
        if line[0].isupper():
          #The line is a header if it matches the required section headers and isn't a complete sentence
          if [item for item in headers_experience if line.lower().startswith(item)] and not checkSentence(line):
            self.header_indices[i]="experience"
          elif [item for item in headers_education if line.lower().startswith(item)] and not checkSentence(line):
            self.header_indices[i]="education"
          elif [item for item in headers_skills if line.lower().startswith(item)] and not checkSentence(line):
            self.header_indices[i]="skills"
          elif [item for item in headers_projects if line.lower().startswith(item)] and not checkSentence(line):
            self.header_indices[i]="projects"

  #Obtain raw text for different resume sections using header indices/linenos
  def getRawResumeSections(self):
    no_lines = len(self.lines)
    header_linenos = list(self.header_indices.keys())
    list_last_index = len(header_linenos)-1
    for counter, lineno in enumerate(header_linenos):
      start_index = lineno+1
      if (counter<list_last_index):
        end_index = header_linenos[counter+1]
      else:
        end_index = no_lines
      self.resume_sections[self.header_indices[lineno]]=self.lines[start_index:end_index]

  def getExperience(self):
    print("getExperience()")

  def getEducation(self):
    print("getEducation()")

  def getSkills(self):
    print("getSkills()")

  #method to parse the entire resume - calls all the above methods of the class
  def getResumeData(self):
    self.getHeaderIndices()
    if len(self.header_indices)!=0:
      self.getRawResumeSections()
      self.getExperience()
      self.getEducation()
      self.getSkills()
    return self.resume_sections

In [21]:
obj = ResumeParser(text)
obj.getResumeData()
print("------------EXPERIENCE---------------------")
for line in obj.resume_sections["experience"]:
  print(line)
print("------------EDUCATION---------------------")
for line in obj.resume_sections["education"]:
  print(line)
print("------------SKILLS---------------------")
for line in obj.resume_sections["skills"]:
  print(line)

getExperience()
getEducation()
getSkills()
------------EXPERIENCE---------------------

Resume Worded, New York, NY
QA Manual Tester

Jun 2018 – Present

● Enabled critical test case complexity metrics with support for Rapid adoption of  functional automation using

a scriptless test case adaptor by standardizing a Test Case construction method that was built
Automation-ready & supported a test automation framework leading to a 45% increase in reusability with
reductions in TCO approaching 25%.

● Optimized scripting, modularity, & maintenance which resulted in an 18% decrease in workflow friction.
● Increased the company’s ability to take and complete projects without increasing manpower by 15% by

reducing QA testing turnaround time by 30%.

Growthsi, New York, NY
QA Manual Tester

Jan 2015 – May 2018

● Restructured utilities & improved the process documentation leading to a 40% reduction in client support

tickets & an 80% increase in uptime.

● Achieved department-wide improvement